# Lamini Classifier Example
First let's install the Lamini Python SDK.

In [2]:
pip install lamini --upgrade


[notice] A new release of pip available: 22.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Now let's import the Lamini Python SDK and set our API key. You can get your API key from app.lamini.ai - every user gets $300 in free credits to get started.

In [58]:
import lamini

lamini.api_key = "a0b00cb22e25401a8228f35c6a89dd68"

Now we're all set up. The first thing we do is define a classifier - Lamini uses the concept of a "project" to contain one or many classifier versions for a given task.

When you create a project, you define several key elements that can't be changed later:

1. A project name
2. A base model for the project to use
3. A set of categories or classes that you need to categorize
4. Descriptions of each category

Projects also (eventually) contain one or many versions of:

1. Example inputs and the expected output - we call this a dataset
2. Models trained on the dataset(s) you've provided


Here we're creating a project named "TriageWorkflowExample" that will leverage the Meta-Llama-3.1-8B-Instruct model.

In [71]:
from lamini.classify.lamini_classifier import LaminiClassifier
import random

cls = LaminiClassifier(
    f"ClassifierExample{random.randint(1000,9999)}"
)

Once the project is created, we define the classes. The more detailed the description, the higher your accuracy will be.

In [72]:

classes = {
    "policy_details": """Chats about policy specifics like coverage, limits, and exclusions""",
    "account_and_billing": """Chats about how the customer pays for their policies or other services""",
}


Adding example inputs is optional, but will also help with accuracy. You can always do this later - we'll show you how later in this notebook.

In [75]:

# policy examples from rows 1-3 of https://docs.google.com/spreadsheets/d/18X_QUlH0G3r2x2epDW3e79TpJsca65gUz9KblHct8a0/edit?gid=1597746069#gid=1597746069
# account_and_billing examples from rows 61-63 of https://docs.google.com/spreadsheets/d/18X_QUlH0G3r2x2epDW3e79TpJsca65gUz9KblHct8a0/edit?gid=1597746069#gid=1597746069

examples = {
    "policy_details": [
       r'''User: Hi, I'm considering purchasing an auto insurance policy from your company. Could you tell me what types of coverage are included?

Bot: Of course! Our standard auto insurance policy includes liability coverage, collision coverage, comprehensive coverage, uninsured/underinsured motorist coverage, and personal injury protection. Is there a specific type of coverage you would like more details about?

User: I'm particularly interested in understanding the comprehensive coverage. What does it entail?

Bot: Comprehensive coverage helps cover damages to your vehicle that aren't caused by a collision, such as theft, vandalism, fire, or natural disasters. It also covers damage from falling objects or an animal striking your vehicle. Does this align with what you're looking for?

User: yes thanks. Can I set up auto payment for this policy?''',
r'''User: Hi, I'm considering getting a new insurance policy and wanted to understand more about the coverage options available.

Bot: Hello! I'd be happy to help. Our policies typically cover areas such as personal injury, property damage, and liability. Is there a specific type of coverage you're interested in?

User: I'm mainly interested in understanding the coverage for property damage. How extensive is it?

Bot: For property damage, our policies cover a wide range of scenarios, including fire, theft, and natural disasters. You can choose different levels of coverage based on your needs. 

User: That's helpful. How do I determine the right level of coverage for my situation?

Bot: Here is the link.

User: Great! I saw the billing choices are limited and payment is always due the firts monday? Do you have auto pay?''',
r'''User: Hi, I’m considering purchasing home insurance and would like to know more about the coverage options you offer.

Bot: Hello! I'd be happy to help with that. Our home insurance policies can cover a range of incidents including natural disasters, theft, and accidental damage. Is there a specific coverage you are interested in?

User: I’m interested in finding a policy that includes both theft and water damage due to plumbing issues. Is that possible?

Bot: Yes, we do offer policies that include coverage for both theft and water damage caused by plumbing issues. Would you like more detailed information on what each of these covers?

User: No. I want to know how much does it cost? What is the regular payment method?''',
    ],
    "account_and_billing": [
        r'''User: Hi, I have a question about my insurance bill. Could you help me understand the charges for this month?

Bot: Of course! Could you please provide your policy number so I can look up your billing details?

User: Sure, my policy number is 12345678.

Bot: Thank you. I see that this month's charges include your annual premium, which is divided into monthly payments. There's also a small fee for the additional roadside assistance coverage you added last month.

User: Ah, I see now. Thank you for explaining that! Is there a way I can view this breakdown online as well?''',
        r'''User: Hi, I'm trying to understand my recent insurance bill. Can you help me figure it out?

Bot: Of course! I'm here to help. What specific part of the bill are you having trouble with?

User: There's a section labeled ""adjustments"" with an additional charge. I don't understand what that means.

Bot: Adjustments often account for changes in your policy, like a change in coverage or a modification in your payment plan. Would you like me to check if any specific modifications were made to your account?

User: Yes, please. I made a change to my deductible last month, could that be it?''',
        r''' User:  Hi, I have a question about my latest insurance bill. Could you help me figure out why it seems higher than usual?

 Bot:  Of course! I'd be happy to help. Could you provide your policy number so I can take a closer look at your billing details?

 User:  Sure, my policy number is 12345678.

 Bot:  Thank you! After reviewing your account, I see that there was an adjustment made this month due to a recent claim. This might explain the increase. Would you like more details on this adjustment?

 User:  Yes, please. I'd like to understand what the adjustment was for and if my bill will be lower next month.'''
    ],
}


Now we initialize the project. This can take about a minute per class, so we'll put in a simple timer to keep us updated on status.

In [74]:
resp = cls.initialize(classes, examples) 

import time

while True:
    print("Waiting for classifier to initialize")
    time.sleep(5)
    resp = cls.train_status()
    if resp["status"] == "completed":
        print("Model ID: " + resp["model_id"])
        first_model_id = resp["model_id"]
        break
    if resp["status"] == "failed":
        print(resp)
        raise Exception("failed training")


Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Model ID: 5eedcc1b-b3f0-44df-a729-a286c0386249


Cool, we have our first model version! Let's try it out with a quick test.

In [77]:
import json

# target: account_and_billing

print(json.dumps(cls.classify(''' User:  Hi there, I have a question about my insurance bill. Can you help me with that?  \n  \n Bot:  Absolutely! I’d be happy to assist you with your billing question. What specifically would you like to know?  \n  \n User:  Could you tell me how I can view and pay my bill online?  \n  \n Bot:  Sure thing! You can view and pay your bill by logging into your account on our website. Once logged in, navigate to the "Billing" section where you\'ll see options to view your statement and make payments using various methods like credit card or bank transfer.  \n  \n User:  Great, but I’m not sure I remember my login details. Can you guide me on how to reset my password?'''), indent=2))


{
  "classification": [
    [
      {
        "class_id": 1,
        "class_name": "account_and_billing",
        "prob": 0.593126710234882
      },
      {
        "class_id": 0,
        "class_name": "policy_details",
        "prob": 0.40687328976511805
      }
    ]
  ]
}


Now we can see how useful the classifier output is. We get a list of all the categories we defined in our project, plus a confidence score for each.

We can go even further to easily quantify the accuracy of our classifier. Let's run an evaluation!

What an evaluation means for a classifier: when you provide a set of inputs and the expected output, we can test the accuracy of the model on those inputs, and give you back both overall metrics as well as per-input assessment. 

In [84]:

# account_and_billing eval
# rows 71-75 from https://docs.google.com/spreadsheets/d/18X_QUlH0G3r2x2epDW3e79TpJsca65gUz9KblHct8a0/edit?gid=1597746069#gid=1597746069
account_eval = [
    {
        "input": r'''User: Hi, can you help me with understanding my insurance bill this month?\n\nBot: Of course! Could you please provide your policy number so I can look up the details?\n\nUser: Sure, my policy number is 123456789.\n\nBot: Thank you! I see your bill includes a premium payment and a small adjustment for a policy change last month. Would you like more details on these charges?\n\nUser: Yes, please explain the adjustment for the policy change.''',
        "target": "account_and_billing"
    },
    {
        "input": r'''User: Hi there! I have a question about my billing statement. \n\nBot: Sure, I\'d be happy to help! What specifically would you like to know about your billing statement?\n\nUser: I noticed an extra charge labeled "service fee" on my last bill. What is this fee for?\n\nBot: The service fee is typically applied for administrative costs related to managing your policy. However, it shouldn\'t be unexpected. Would you like me to check if it\'s a recurring charge on your plan?\n\nUser: Yes, please! I\'d appreciate that.''',
        "target": "account_and_billing"
    },
    {
        "input": r'''User: Hi, I\'m trying to understand why my premium this month seems higher than usual. Can you help with that?\n\nBot: Of course! There could be a few reasons for an increase in your premium. Have you recently updated your policy or added any new coverage?\n\nUser: Hmm, I don\'t think I\'ve made any changes. Could it be something else?\n\nBot: It\'s possible that there were changes in your area that affected your rate or adjustments made at the time of your policy renewal. I can also check if any discounts have expired.\n\nUser: That would be great! Could you take a look at whether any discounts I had have been removed?''',
        "target": "account_and_billing"
    },
    {
        "input": r'''User:  Hi, I got a question about my insurance billing. Could you help me with how I can check my current payment status?\n\n Bot:  Certainly! To check your current payment status, you can log into your account on our website or use our mobile app. Once logged in, navigate to the "Billing" section, and you should be able to view your current status and any upcoming payments.\n\n User:  What if I don\'t have access to the internet right now? Is there another way to check?\n\n Bot:  Of course! You can check your payment status by calling our customer service hotline. Our representatives will be happy to assist you with the information you need.\n\n User:  Great, thank you! I\'ll give them a call.''',
        "target": "account_and_billing"
    },
    {
        "input": r'''User: Hi, I\'m trying to understand how to set up automatic payments for my insurance bills. Can you help?\n\nBot: Of course! You can set up automatic payments through our online portal. Would you like instructions on how to do that?\n\nUser: Yes, please. That would be great.\n\nBot: Once you log into your account on our website, click on "Billing & Payments" and then select "Set Up Automatic Payments." You\'ll be guided to enter your bank details and confirm your payment schedule.\n\nUser: Perfect, thank you! I\'ll do that now.''',
        "target": "account_and_billing"
    }
]

# policy_details eval
# rows 1-5 from https://docs.google.com/spreadsheets/d/18X_QUlH0G3r2x2epDW3e79TpJsca65gUz9KblHct8a0/edit?gid=1597746069#gid=1597746069
policy_eval = [
    {
        "input": r'''User: Hi, I\'m considering purchasing an auto insurance policy from your company. Could you tell me what types of coverage are included?\n\nBot: Of course! Our standard auto insurance policy includes liability coverage, collision coverage, comprehensive coverage, uninsured/underinsured motorist coverage, and personal injury protection. Is there a specific type of coverage you would like more details about?\n\nUser: I\'m particularly interested in understanding the comprehensive coverage. What does it entail?\n\nBot: Comprehensive coverage helps cover damages to your vehicle that aren\'t caused by a collision, such as theft, vandalism, fire, or natural disasters. It also covers damage from falling objects or an animal striking your vehicle. Does this align with what you\'re looking for?\n\nUser: yes thanks. Can I set up auto payment for this policy?''',
        "target": "policy_details"
    },
    {
        "input": r'''User: Hi, I\'m considering getting a new insurance policy and wanted to understand more about the coverage options available.\n\nBot: Hello! I\'d be happy to help. Our policies typically cover areas such as personal injury, property damage, and liability. Is there a specific type of coverage you\'re interested in?\n\nUser: I\'m mainly interested in understanding the coverage for property damage. How extensive is it?\n\nBot: For property damage, our policies cover a wide range of scenarios, including fire, theft, and natural disasters. You can choose different levels of coverage based on your needs. \n\nUser: That\'s helpful. How do I determine the right level of coverage for my situation?\n\nBot: Here is the link.\n\nUser: Great! I saw the billing choices are limited and payment is always due the firts monday? Do you have auto pay?''',
        "target": "policy_details"
    },
    {
        "input": r'''User: Hi, I’m considering purchasing home insurance and would like to know more about the coverage options you offer.\n\nBot: Hello! I\'d be happy to help with that. Our home insurance policies can cover a range of incidents including natural disasters, theft, and accidental damage. Is there a specific coverage you are interested in?\n\nUser: I’m interested in finding a policy that includes both theft and water damage due to plumbing issues. Is that possible?\n\nBot: Yes, we do offer policies that include coverage for both theft and water damage caused by plumbing issues. Would you like more detailed information on what each of these covers?\n\nUser: No. I want to know how much does it cost? What is the regular payment method?''',
        "target": "policy_details"
    },
    {
        "input": r'''User: Hi, I was considering getting a home insurance policy and wanted to know what kind of coverage you offer.\n\nBot: Hello! We offer comprehensive coverage that includes protection for your home structure, personal belongings, liability, and additional living expenses in case of a covered loss. Is there anything specific you would like more details about?\n\nUser: That\'s great. Could you tell me more about the length of the policy term?\n\nBot: Certainly! Our standard home insurance policies typically have terms of one year, with options to renew annually. We also offer discounts for customers who choose to sign up for multiple years. Does this work for you?\n\nUser: Can I pay through online portal for the premium? Or what alternative payment method do you have?''',
        "target": "policy_details"
    },
    {
        "input": r'''User: Hi there! I\'m considering getting a health insurance policy with your company and I was wondering about the coverage details.\n\nBot: Hello! I\'d be happy to assist with that. Our health insurance policies typically cover doctor visits, hospital stays, prescription medications, and preventive care. Are there specific areas of coverage you\'re interested in?\n\nUser: Great, what about coverage for specialist visits, such as seeing a dermatologist or cardiologist?\n\nBot: Yes, our policies do cover specialist visits. It usually requires a referral from your primary care doctor, but once you have that, you\'re covered for specialist consultations as well.\n\nUser: Fine. What is usually the due date of the premium?''',
        "target": "policy_details"
    }
]

from lamini.one_evaler.one_evaler import LaminiOneEvaler

eval = LaminiOneEvaler(
    test_model_id=first_model_id,
    eval_data_id=f"first_eval{random.randint(1000,9999)}",
    eval_data=account_eval+policy_eval,
    test_eval_type="classifier",
)

full_eval = eval.run()

print(json.dumps(full_eval, indent=2))

def print_missed_evals(eval_results_str):
    try:
        eval_results = json.loads(eval_results_str)
        for result in eval_results:
            test_output = result['prediction']
            target = result['target']
            
            if test_output != target:
                print(f"\nMissed Evaluation")
                print(f"Expected: {target}")
                print(f"Predicted: {test_output}")
                print(f"Input: {result['input']}")
                print(f"Confidence: {result.get('confidence', 'N/A')}")
                print("-" * 80)
    except json.JSONDecodeError:
        print("Error: Could not parse evaluation results as JSON")
    except TypeError as e:
        print(f"Error processing results: {e}")
        print(f"Raw results: {eval_results_str}")

print_missed_evals(full_eval)

{
  "eval_job_id": "394282949",
  "eval_data_id": "first_eval9982",
  "metrics": {
    "tuned_accuracy": 0.9,
    "tuned_precision": 1.0,
    "tuned_recall": 0.9,
    "tuned_f1": 0.9473684210526315
  },
  "status": "COMPLETED",
  "predictions": [
    {
      "input": "User: Hi, can you help me with understanding my insurance bill this month?\\n\\nBot: Of course! Could you please provide your policy number so I can look up the details?\\n\\nUser: Sure, my policy number is 123456789.\\n\\nBot: Thank you! I see your bill includes a premium payment and a small adjustment for a policy change last month. Would you like more details on these charges?\\n\\nUser: Yes, please explain the adjustment for the policy change.",
      "target": "account_and_billing",
      "test_output": "policy_details",
      "base_output": null
    },
    {
      "input": "User: Hi there! I have a question about my billing statement. \\n\\nBot: Sure, I\\'d be happy to help! What specifically would you like to know 

Best practice: after you've added a few high-quality examples, you should run an eval and carefully review the ground truth labels to make sure they're aligned with the classifier's task and scope - the tags in a general eval set aren't always the best for a narrowly-defined classifier agent to learn from. Don't just review inputs where the assigned class was completely wrong - also review inputs where the classifier's answer is correct but the confidence score is low.

Even with just 3 examples of each class, we already got 9 of 10 eval inputs correct. Let's go for 100% accuracy!

We should look at the content of the missed input(s), source or generate examples that cover the relevant concepts or structure of the missed input(s), and add them to the classifier.

Above, the missed input was

In [65]:

## TODO update

resp = cls.add(
    "additional_data",
    {
        "account_and_billing": [
            "Cats spend up to sixteen hours a day sleeping, making them some of nature's most dedicated nappers.",
            "Felines possess an extraordinary sense of balance thanks to their flexible backbone and righting reflex.",
            "A cat's sandpaper-like tongue is covered in tiny hooks called papillae that help them groom themselves effectively.",
            "Female cats tend to be right-pawed while male cats are more often left-pawed, according to scientific studies.",
            "Ancient Egyptians showed their devotion to cats by mummifying them alongside their human companions.",
        ],
        "policy_details": [
            "Dogs have evolved alongside humans for over 15,000 years, developing an uncanny ability to read our facial expressions and emotions.",
            "The average dog can understand around 165 different words or signals, though some exceptional dogs can learn many more.",
            "A dog's sense of smell is roughly 40 times greater than a human's, allowing them to detect diseases and track scents that are days old.",
            "Unlike humans who have three cones in their eyes, dogs only have two, making them partially colorblind but excellent at detecting movement.",
            "The Basenji breed is known as the 'barkless dog' because it produces an unusual yodel-like sound instead of a typical bark.",
        ],
    },
)

resp = cls.train()

while True:
    print("Waiting for classifier to train")
    time.sleep(5)
    resp = cls.train_status()
    if resp["status"] == "completed":
        print("Model ID: " + resp["model_id"])
        second_model_id = resp["model_id"]
        break
    if resp["status"] == "failed":
        print(resp["status"])
        raise Exception("failed training")


Waiting for classifier to train
Waiting for classifier to train
Model ID: 6f82d155-2d1d-4bfe-9cc1-6824c6aaddf3


Great, now we have a second model version in our project! Let's run an eval and compare it to the first version.

In [68]:
print("Running comparison eval between model versions " + first_model_id + " and " + second_model_id)

eval_2 = LaminiOneEvaler(
    test_model_id=first_model_id,
    eval_data_id=f"second_eval{random.randint(1000,9999)}",
    eval_data=[{"input": "woof", "target": "dog"}, {"input": "meow", "target": "cat"}],
    test_eval_type="classifier",
    base_model_id=second_model_id,
    sbs=True,
    fuzzy=True,
)

print(json.dumps(eval_2.run(), indent=2))

Running comparison eval between model versions bfa01a54-041a-49d5-af27-4f5f5b808177 and 6f82d155-2d1d-4bfe-9cc1-6824c6aaddf3
{
  "eval_job_id": "984315324",
  "eval_data_id": "second_eval8620",
  "metrics": {
    "base_accuracy": 0.0,
    "base_precision": 0.0,
    "base_recall": 0.0,
    "base_f1": 0.0,
    "base_fuzzy_accuracy": 0.0,
    "base_fuzzy_precision": 0.0,
    "base_fuzzy_recall": 0.0,
    "base_fuzzy_f1": 0.0,
    "tuned_accuracy": 0.0,
    "tuned_precision": 0.0,
    "tuned_recall": 0.0,
    "tuned_f1": 0.0,
    "tuned_fuzzy_accuracy": 0.0,
    "tuned_fuzzy_precision": 0.0,
    "tuned_fuzzy_recall": 0.0,
    "tuned_fuzzy_f1": 0.0,
    "tuned_win_loss_ratio": 0.0,
    "base_win_loss_ratio": 0.0
  },
  "status": "COMPLETED",
  "predictions": [
    {
      "input": "woof",
      "target": "dog",
      "test_output": "account_and_billing",
      "base_output": "policy_details"
    },
    {
      "input": "meow",
      "target": "cat",
      "test_output": "policy_details",
  

The eval output makes it easy to compare model versions overall, and to see exactly where the differences are, so you know exactly where to focus to improve your workflow.

Happy classifying!